In [ ]:
# pip install pandas mysql-connector-python
# pip install openpyxl
# pip install python-dotenv

import pandas as pd
import mysql.connector
from mysql.connector import Error

# --- DB 연결 정보 ---
DB_CONFIG = {
    'host': '210.121.189.12',       # 예: 144.24.79.229 또는 localhost
    'user': 'remoteuser',
    'password': 'deepdish0000',
    'database': 'deep_dish'
}

db_conn = mysql.connector.connect(**DB_CONFIG)
cursor = db_conn.cursor()
print("MySQL DB에 성공적으로 연결되었습니다.")


MySQL DB에 성공적으로 연결되었습니다.


In [ ]:
 # 1. DB에서 menus와 ingredients 정보를 미리 가져와 딕셔너리로 만듭니다. (효율성 UP)
cursor.execute("SELECT menu_id, menu_name FROM menus")
menu_map = {name: id for id, name in cursor.fetchall()}

cursor.execute("SELECT ingredient_id, ingredient_name FROM ingredients")
ingredient_map = {name: id for id, name in cursor.fetchall()}
print("메뉴 및 식재료 ID 맵을 생성했습니다.")

# 2. Pandas로 CSV 파일을 읽습니다.
CSV_FILE_PATH = '음식별 식재료.xlsx'
df = pd.read_excel(CSV_FILE_PATH)

메뉴 및 식재료 ID 맵을 생성했습니다.


UnicodeDecodeError: 'cp949' codec can't decode byte 0xee in position 15: illegal multibyte sequence

In [ ]:

# --- 파일 정보 ---
CSV_FILE_PATH = '음식별 식재료.xlsx'

def populate_menu_ingredients():
    db_conn = None
    cursor = None
    try:
        db_conn = mysql.connector.connect(**DB_CONFIG)
        cursor = db_conn.cursor()
        print("MySQL DB에 성공적으로 연결되었습니다.")

        # 1. DB에서 menus와 ingredients 정보를 미리 가져와 딕셔너리로 만듭니다. (효율성 UP)
        cursor.execute("SELECT menu_id, menu_name FROM menus")
        menu_map = {name: id for id, name in cursor.fetchall()}
        
        cursor.execute("SELECT ingredient_id, ingredient_name FROM ingredients")
        ingredient_map = {name: id for id, name in cursor.fetchall()}
        print("메뉴 및 식재료 ID 맵을 생성했습니다.")

        # 2. Pandas로 CSV 파일을 읽습니다.
        df = pd.read_csv(CSV_FILE_PATH)
        # CSV의 첫 번째 컬럼 이름을 'ingredient_name'으로 가정합니다.
        ingredient_col_name = df.columns[0]
        df.rename(columns={ingredient_col_name: 'ingredient_name'}, inplace=True)
        print("CSV 파일을 성공적으로 읽었습니다.")

        # 3. INSERT할 데이터를 담을 리스트를 생성합니다.
        data_to_insert = []
        
        # 4. CSV 데이터를 순회하며 menu_id와 ingredient_id의 조합을 찾습니다.
        for menu_name in df.columns[1:]: # 첫 번째(식재료) 열 제외
            if menu_name in menu_map:
                menu_id = menu_map[menu_name]
                
                # 해당 메뉴 열(column)에서 값이 있는(X표시 등) 식재료를 찾습니다.
                used_ingredients = df[df[menu_name].notna()]['ingredient_name']
                
                for ingredient_name in used_ingredients:
                    if ingredient_name in ingredient_map:
                        ingredient_id = ingredient_map[ingredient_name]
                        # (menu_id, ingredient_id) 튜플을 리스트에 추가
                        data_to_insert.append((menu_id, ingredient_id))
                    else:
                        print(f"[경고] DB에 없는 식재료: {ingredient_name}")
            else:
                print(f"[경고] DB에 없는 메뉴: {menu_name}")

        # 5. executemany를 사용해 DB에 데이터를 한 번에 INSERT 합니다.
        if data_to_insert:
            sql_insert = "INSERT IGNORE INTO menu_ingredients (menu_id, ingredient_id) VALUES (%s, %s)"
            cursor.executemany(sql_insert, data_to_insert)
            db_conn.commit()
            print(f"총 {cursor.rowcount}개의 메뉴-식재료 관계를 DB에 저장했습니다.")
        else:
            print("DB에 저장할 데이터가 없습니다.")

    except Error as e:
        print(f"DB 작업 중 에러 발생: {e}")
    finally:
        if db_conn and db_conn.is_connected():
            cursor.close()
            db_conn.close()
            print("MySQL 연결이 해제되었습니다.")

# --- 스크립트 실행 ---
if __name__ == '__main__':
    populate_menu_ingredients()